# KoBigBird_Keyword_Extraction_Frequancy_Weighted

기존 방법에 빈도수에 따른 Frequancy_Weightt를 반영하는 모델

## 본문 키워드 형태소 분석하여 후보 키워드 추출하기

형태소 분석기 komoran을 사용하기 위해 konlpy 설치

In [1]:
#!pip install konlpy

### 불용어 제거 함수 정의

In [58]:
def remove_stopwords(nouns, stopwords):
    for n in nouns:
        if n in stopwords:
            nouns.remove(n)
    return nouns

영어 불용어 받아오기

In [41]:
from nltk.corpus import stopwords as eng_stopwords

eng_stopwords_set = set(eng_stopwords.words('english')) 

### 영어 단어 명사 추출 함수 정의

In [37]:
import nltk

from string import punctuation
import re

def get_nouns_eng(text):

    # 명사 추출 시 url 제거
    url_pattern = re.compile(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))")
    text = re.sub(url_pattern, '', text)
    
    text = text.encode('utf-8').decode('ascii', 'ignore')

    result = ""
    for t in text:
        if t not in punctuation:
            result += t

    text = result.strip()

    word_tokens = nltk.word_tokenize(text)
    tokens_pos = nltk.pos_tag(word_tokens)
    NN_words = []
    for word, pos in tokens_pos:
        if 'NN' in pos or 'NNP' in pos:
            NN_words.append(word)
            
    return NN_words

## 불용어(stopwords) 사전 정의

두 개의 한국어 불용어 사전을 이용

In [51]:
import csv

stopwords = []
with open('./data/stopwords/stopwords.txt', 'r') as f:
    for line in f:
        stopwords.append(line.rstrip('\n'))

with open('./data/stopwords/stopwords-ko.txt', 'r') as f:
    for line in f:
        stopwords.append(line.rstrip('\n'))
        
with open('./data/stopwords/stopwords-np.txt', 'r') as f:
    tsv_file = csv.reader(f, delimiter="\t")
    for line in tsv_file:
        stopwords.append(line[0])

stopwords_set = set(stopwords)
print(stopwords_set)

{'할지언정', '남들', '저희', '뒤이어', '다음으로', '알았어', '하물며', '더라도', '기준으로', '아하', '에', '끼익', '함께', '텐', '것', '도달하다', '이지만', '월', '이렇구나', '한다면 몰라도', '그러나', '=', '탕탕', '４', '예', '가지', '얼마', '기대여', '비걱거리다', '편', '》', '시각', '１', '만큼', '매번', '놀라다', '아무도', '가령', '대해서', '여보시오', '기점으로', '고로', '대로 하다', '하려고하다', '>', '않', '개의치않고', '힘입어', '뿐이다', '같이', '우리들', '뿐', '-', '인젠', '어떤것들', '운운', '다음', '도착하다', '총적으로 말하면', '펄렁', '?', '어찌', '설사', '아니나다를가', '즉시', '개', '더욱이는', '륙', '하도다', '허', '그렇게 함으로써', '관해서는', '어쨋든', '된바에야', '만이 아니다', '영', '이와 같다', '총적으로', '이천구', '훨씬', '까지 미치다', '따르', '그위에', '앞에서', '곳', '인', '중의하나', '｛', '쳇', '같', '소생', '이렇게 많은 것', '어떻', '버', '저것만큼', '어느것', '비하면', '그때', '삐걱거리다', '？', '등등', '나머지는', '대해 말하자면', '입장에서', '급', '어찌됏어', '1', '자', '이외에도', '어느때', '의해되다', '하든지', '연이서', '주', '어느해', '이렇', '거의', '의', '더욱더', '생각하', '＞', '그들', ',', '와아', '——', '한켠으로는', '지든지', '아이고', '향해서', '노', '차라리', '.', '영차', '중에서', '하는 김에', '이봐', '하기보다는', '：', '모르', '중', '옆사람', '반대로 말하자면', '우르르', '까악', '아니었다면', '만들', '아이야', '～',

### 각 형태소 분석기별 명사 추출 함수 정의

형태소 분석을 통해 명사 추출, 불용어 제거, 영어 단어 추출하는 함수

In [78]:
from konlpy.tag import Komoran
from konlpy.tag import Kkma
from konlpy.tag import Okt
from collections import Counter

def get_nouns_komoran(text):
    komoran = Komoran()
    nouns = komoran.nouns(text)
    eng_nouns = get_nouns_eng(text)
    eng_nouns = [ n for n in eng_nouns if len(n) != 1 ]
    eng_nouns = remove_stopwords(eng_nouns, eng_stopwords_set)
    nouns.extend(eng_nouns)
    counts = Counter(nouns)
    nouns = set(nouns)
    nouns = list(nouns)
    nouns = remove_stopwords(nouns, stopwords_set)
    
    return nouns, counts

def get_nouns_kkma(text):
    kkma = Kkma()
    nouns = kkma.nouns(text)
    nuons = remove_stopwords(nouns, stopwords_set)
    nouns = set(nouns)
    nouns = list(nouns)
    eng_nouns = get_nouns_eng(text)
    eng_nouns = [ n for n in eng_nouns if len(n) != 1 ]
    nouns.extend(eng_nouns)
    return nouns

def get_nouns_okt(text):
    okt = Okt()
    nouns = okt.nouns(text)
    nuons = remove_stopwords(nouns, stopwords_set)
    nouns = set(nouns)
    nouns = list(nouns)
    eng_nouns = get_nouns_eng(text)
    eng_nouns = [ n for n in eng_nouns if len(n) != 1 ]
    nouns.extend(eng_nouns)
    
    return nouns

In [71]:
# sample_nouns = get_nouns_komoran(sample[1]['body_text'])
# print(sample[1]['title'])
# print(sample[1]['body_text']+'\n')
# print(sample_nouns)

### 문장to문장 sts finetuning된 KoBigBird 모델 불러오기

In [72]:
from transformers import  AutoModel,AutoTokenizer
model = AutoModel.from_pretrained("./model/saved_model_epoch_79.pt")
tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base")

Some weights of BigBirdModel were not initialized from the model checkpoint at ./model/saved_model_epoch_79.pt and are newly initialized: ['pooler.weight', 'pooler.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 데이터 불러오기

In [73]:
import jsonlines

sample = []

with jsonlines.open("./data/brunch/brunch_dataset_100.jsonl") as f:
    for line in f.iter():
        sample.append(line)

In [74]:
sample[0]

{'title': '구글을 대체할 AI 챗봇 ChatGPT',
 'sub_title': '브런치의 미래\xa0',
 'body_text': '\n일론머스크와 샘알트만이 설립한 OpenAI에서 ChatGPT이라는 대화형 인공지능 챗봇을 선보였습니다. ChatGPT은 사용자와 주고받는 대화에서 질문에 답하도록 설계되어 있습니다. GPT는 질문에 답하는 것은 물론 정확하지 않은 전제에 대해 이의를 제기하기도 하고 부적절한 요청을 거부하는 등 마치 인격을 가진 인간과의 대화 구현이 가능합니다. \n\n이러한 GPT가 주목받게 된 것은 그 기능 자체의 뛰어남도 있지만 글로벌 기업인 구글의 모회사 CEO가 직접 언급하며 급속도로 기사화되었습니다. 구글에서 GPT를 경계하는 이유는 장기적으로 검색 엔진을 대체할 수 있을 것이라 염려 때문입니다. 현재의 검색 엔진은 사용자가 직접 검색 대상을 문법에 맞게 지시 내려야 합니다. 그러나 GPT는 대화형이기 때문에 누구나 손쉽게 이용할 수 있으며 더 다양한 정보를 (데이터)를 확보할 수 있습니다. \n\n제가 GPT에 관심을 갖게 된 것은 바로 GPT로 뛰어난 글을 작성할 수 있다는 부분 때문이었습니다. 실제로 해외 사용자들의 후기를  통해 GPT를 이용해 소설과 시를 썼다는 후기들을 손쉽게 볼 수 있습니다. \n*GPT는 영어로 커뮤니케이션하는 것이 더 원활합니다. 한국어도 지원하나 정보가 적으며 구동이 느립니다.\n\n캐나다 언론인이자 임상 심리학자이자인 조던피터슨(Jordan Bernt Peterson)은 GPT의 위험성에 대해 이야기하였습니다. 그의 날카로운 지적과 더불어 해당 영상을 본 사람들의 반응 또한 높은 통찰을 보여주고 있습니다. (자세한 내용은 아래 연결된 링크에서 확인 가능합니다)\n\n과거 인터넷의 발달로 우리는 편리한 생활을 누리게 되었지만 그만큼 축소된 부분도 존재합니다. 그 부분은 바로 생각에 대한 영역입니다. 세계적인 경영컨설턴트이자 IT 미래학자인 니콜라스 카가 그의 저서 <생각하지 않는 사람들>에서

In [75]:
import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"

## 최종 키워드 추출 함수(빈도수에 따른 가중치 반영)

In [93]:
def extract_main_keywords(nouns, counts, title, text, model, tokenizer, use_freq_w=False):
    
    tokenized_nouns = {}
    for keyword in nouns:
        tokenized_nouns[keyword] = tokenizer(keyword, max_length=32, padding='max_length', return_tensors='pt')
    
    keyword_vectors = {}
    for keyword, tokenized in tokenized_nouns.items():
        output = model(**tokenized.to(device))[0][:,0,:]
        keyword_vectors[keyword] = output
        
    tokenized_title = tokenizer(title, max_length=128, padding="max_length", return_tensors='pt')
    tokenized_text = tokenizer(text, max_length=4096, padding="max_length", return_tensors='pt')

    title_vec = model(**tokenized_title.to(device))[0][:,0,:]
    body_text_vec = model(**tokenized_text.to(device))[0][:,0,:]
    
    if use_freq_w:
        freq_w = {}
        total_candidate_words = len(nouns)
        for keyword in nouns:
            freq_w[keyword] = counts[keyword]/total_candidate_words
         
        for keyword, vec in keyword_vectors.items():
            body_text_vec = ((freq_w[keyword]* vec) + body_text_vec)
        
        body_text_vec = body_text_vec/total_candidate_words
        
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    similarity = {}

    for keyword, vector in keyword_vectors.items():
        sim = cos(vector,  title_vec*(0.2) + body_text_vec*(0.8))
        similarity[keyword] = sim
        
    sorted_dict = sorted(similarity.items(), key = lambda item: item[1], reverse = True)
    
    # print(keyword_vectors)
    print(f"제목과 본문의 유사도 {cos(title_vec, body_text_vec)}\n")
    print(title+'\n')
    print(text+'\n')
    print(sorted_dict,'\n')
    print(nouns)

In [96]:
model.to('cuda')
model.eval()

sample_essay = sample[1]

with torch.no_grad():
    print(sample_essay['keyword'])
    
    title = sample_essay['title']
    text = sample_essay['body_text']
    text = text.replace('\n','').replace('\t','').replace('\r','') # 코모란 에러 방지
    nouns, counts = get_nouns_komoran(text)
    print(counts)
    extract_main_keywords(nouns, counts, title, text, model, tokenizer, use_freq_w=True)

['AI', '예술', 'NFT']
Counter({'AI': 22, '기술': 7, '작품': 7, '예술': 7, '것': 7, '수': 6, '디지털': 5, '발전': 4, '학습': 4, '논란': 4, '아트': 4, '지적': 4, '재산권': 4, '보호': 4, '분야': 3, '창작': 3, '년': 3, '이미지': 3, '미술': 3, '그림': 3, '해당': 3, '아티스트': 3, '미국': 3, '개인정보': 3, '개정': 3, '산업': 2, '활용': 2, '이제': 2, '후': 2, '문학': 2, '시아': 2, '시': 2, '콜로라도': 2, '대회': 2, '부문': 2, '제작': 2, '.kr': 2, '앨런': 2, '이야기': 2, '이후': 2, '미학': 2, '사람': 2, '등': 2, '영역': 2, '흐름': 2, '데이터': 2, '방식': 2, '예술가': 2, '발': 2, '콘셉트': 2, '협회': 2, '법률': 2, '데': 2, '기업': 2, '윤리': 2, '강령': 2, '그동안': 1, '문화예술': 1, '모습': 1, '지속': 1, '콘텐츠': 1, '생산': 1, '사고력': 1, '판단력': 1, '학습력': 1, '소프트웨어': 1, '상황': 1, '판단': 1, '해결책': 1, '제안': 1, '능력': 1, '과거': 1, '모방': 1, '수준': 1, '대표': 1, '4월': 1, '개발': 1, '달리': 1, '화풍': 1, '독창': 1, '시각': 1, '진출': 1, '8월': 1, '시인': 1, '시집': 1, '출간': 1, '편': 1, '작법': 1, '종합': 1, '문체': 1, '집필': 1, '현재': 1, '각종': 1, '도전': 1, '습작': 1, '상반기': 1, '중심': 1, '다음': 1, '달인': 1, '9월': 1, '주립': 1, '박람회': 1, '수상작': 1, '세간': 1, '주목': 1, '제품': 

구현해야 할 점

* 단어를 배치단위로 입력하여 한번에 연산할 수 있게 변형
* 빈도수에 따른 가중치 변환
* 제목-본문 STS/NLI fine-tuning
* 단어-제목 STS/NLI fine-tuning
* 단어-본문 STS/NLI fine-tuning

### kr-wordrank와 비교

In [84]:
from krwordrank.word import KRWordRank

min_count = 5   # 단어의 최소 출현 빈도수 (그래프 생성 시)
max_length = 10 # 단어의 최대 길이
wordrank_extractor = KRWordRank(min_count=min_count, max_length=max_length)

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10
texts = [text]
keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)

for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
        print('%8s:\t%.4f' % (word, r))

      사용:	2.3623
      것이:	2.3267
      되면:	2.2527
    서비스를:	2.1354
      되는:	2.0558
    계속해서:	1.7597
     습관이:	1.5846
     있다.:	1.4460
      많은:	1.3472
     가변적:	1.2662
      있는:	1.2482
      내부:	1.0884
      계기:	1.0522
      보상:	0.9734
      아주:	0.7625
      우리:	0.7340
      유도:	0.6099
      대뇌:	0.5950
      외부:	0.5934
      행동:	0.5179
      형성:	0.4879


## 명사구 추출 코드

In [254]:
komoran = Komoran()
pos = komoran.pos(text)
print(pos)

[('“', 'SS'), ('미시령', 'NNP'), ('고개', 'NNG'), ('를', 'JKO'), ('넘어가', 'VV'), ('자', 'EC'), ('눈앞', 'NNG'), ('에', 'JKB'), ('펼쳐지', 'VV'), ('ㄴ', 'ETM'), ('풍경', 'NNG'), ('을', 'JKO'), ('보', 'VV'), ('며', 'EC'), ('써니', 'NNP'), ('가', 'JKS'), ('말', 'NNG'), ('하', 'XSV'), ('았', 'EP'), ('어', 'EF'), ('.', 'SF'), ('”', 'SS'), ('<', 'SS'), ('와', 'IC'), ('!', 'SF'), ('울산', 'NNP'), ('바지', 'NNP'), ('다', 'EF'), ('!', 'SF'), ('>', 'SS'), ('일명', 'NNG'), ('울산', 'NNP'), ('바지', 'NNP'), ('사건', 'NNG'), ('은', 'JX'), ('30', 'SN'), ('년', 'NNB'), ('이', 'JKS'), ('다', 'MAG'), ('다', 'MAG'), ('되', 'VV'), ('어', 'EC'), ('가', 'VX'), ('는', 'ETM'), ('지금', 'NNG'), ('까지', 'JX'), ('도', 'JX'), ('동생', 'NNG'), ('이', 'JKS'), ('올케', 'NNG'), ('를', 'JKO'), ('놀리', 'VV'), ('ㄹ', 'ETM'), ('때', 'NNG'), ('사골', 'NNP'), ('처럼', 'JKB'), ('우려먹', 'VV'), ('는', 'ETM'), ('단골', 'NNG'), ('레퍼토리', 'NNG'), ('다', 'JX'), ('.', 'SF'), ('그', 'MM'), ('때', 'NNG'), ('동생', 'NNG'), ('과', 'JC'), ('올케', 'NNG'), ('모두', 'MAG'), ('아름답', 'VA'), ('ㄴ', 'ETM'), ('삼십', 'NR'), 

In [272]:
komoran = Komoran()
pos = komoran.pos(text)

np_list = []
np = ''
cnt = 0
for tup in pos:
    if 'N' in tup[1]:
        np = np + tup[0]
        cnt += 1
        
    elif np != '' and cnt != 1:
        np_list.append(np)
        np = ''
        cnt = 0
    else:
        np = ''
        cnt = 0

np_list

['미시령고개',
 '울산바지',
 '일명울산바지사건',
 '30년',
 '때사골',
 '단골레퍼토리',
 '때동생',
 '삼십대',
 '말실수',
 '모르나동생',
 '어휘실수',
 '식사자리',
 '내동생',
 '말실수',
 '대인배로',
 '요즘내증세이',
 '거기그것옆',
 '송년가족모임준비상황',
 '요리대첩',
 '내담당',
 '튀김젓가락',
 '주변말',
 '상황사람',
 '단어하나',
 '사람이름',
 '영화제목',
 '소설속',
 '주인공모친',
 '당신안경',
 '일본인경호대장역',
 '박뭐',
 '박해영']